In [1]:


!pip install -q /kaggle/input/rsna-bcd-whl-ds/python_gdcm-3.0.20-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -q /kaggle/input/rsna-bcd-whl-ds/pylibjpeg-1.4.0-py3-none-any.whl
!pip install -q /kaggle/input/rsna-bcd-whl-ds/dicomsdl-0.109.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl




In [2]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import tqdm
import dicomsdl as dicom


In [3]:
train = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/train.csv')
test = pd.read_csv('/kaggle/input/rsna-breast-cancer-detection/test.csv')
train.head()
train = train[[col for col in train.columns if col in ['site_id',
 'patient_id',
 'image_id',
 'laterality',
 'view',
 'age',
 'implant',
 'machine_id',
 'prediction_id','cancer']]]
train.loc[train['age'].isnull(),'age'] = train['age'].mean()

#loading directory of images
path = os.path.join('/kaggle/input/rsna-breast-cancer-detection/test_images')
list_dir = os.listdir(path)
k =[]
y={}
path_list=[]
mlo_path_l = []
cc_path_l = []
mlo_path_r = []
cc_path_r = []
for i, sub in tqdm.tqdm(enumerate(list_dir)):
    for j, fname in enumerate(os.listdir(os.path.join(path,sub))):
        view = (test.loc[test['image_id']==int(((fname).split('.'))[0]),'view'].values)[0]
        laterality = (test.loc[test['image_id']==int(((fname).split('.'))[0]),'laterality'].values)[0]
        k.append(os.path.join(os.path.join(path,sub),fname))
        if view =='MLO':
            if laterality=='L':
                mlo_path_l.append(os.path.join(os.path.join(path,sub),fname))
                path_list.append(os.path.join(os.path.join(path,sub),fname))
                y[sub+'_{}_'.format(laterality)+view] = (os.path.join(os.path.join(path,sub),fname))
            else:
                mlo_path_r.append(os.path.join(os.path.join(path,sub),fname))
                path_list.append(os.path.join(os.path.join(path,sub),fname))
                y[sub+'_{}_'.format(laterality)+view] = (os.path.join(os.path.join(path,sub),fname))
        elif view =='CC':
            if laterality=='L':
                cc_path_l.append(os.path.join(os.path.join(path,sub),fname))
                path_list.append(os.path.join(os.path.join(path,sub),fname))
                y[sub+'_{}_'.format(laterality)+view] = (os.path.join(os.path.join(path,sub),fname))
            else:
                cc_path_r.append(os.path.join(os.path.join(path,sub),fname))
                path_list.append(os.path.join(os.path.join(path,sub),fname))
                y[sub+'_{}_'.format(laterality)+view] = (os.path.join(os.path.join(path,sub),fname))
mlo_path_dict_l={}
for i in range(len(list(mlo_path_l))):
    mlo_path_dict_l[i] = mlo_path_l[i]
mlo_path_dict_r={}
for i in range(len(list(mlo_path_r))):
    mlo_path_dict_r[i] = mlo_path_r[i]
cc_path_dict_l={}
for i in range(len(list(cc_path_l))):
    cc_path_dict_l[i] = cc_path_l[i]
cc_path_dict_r={}
for i in range(len(list(cc_path_r))):
    cc_path_dict_r[i] = cc_path_r[i]   
path_dict={}
for i in range(len(list(path_list))):
    path_dict[i] = path_list[i] 
dict_path = {}
for i,j in y.items():
    dict_path[j] = i.split('_')[1]
d_path = {}
check={}
for i,j in enumerate((y.keys())):
    check[j] = i 
for i,j in enumerate(y.keys()):
    d_path[j.split('_')[0]+'_'+j.split('_')[1]] = 0

1it [00:00, 147.59it/s]


In [4]:
tf.config.run_functions_eagerly(True)
@tf.function
def gen(x):
    i = path_dict[x]
    image = (dicom.open(i).pixelData(storedvalue=True))
    max_val = tf.cast(image[-1,1000],dtype=tf.float32)
    #print(max_val)
    la = dict_path[i]
    
    #if  train[train['image_id']==int(i.split('/')[-1].split('.')[0])]['laterality'].values[0]=='L':
    if la == 'L':
        if max_val>=1000:
            return (max_val-tf.image.resize(tf.expand_dims(image[100:-100,:2500],axis=-1),[150,100],preserve_aspect_ratio=False))/255.0
        else:
            return (tf.image.resize(tf.expand_dims(image[100:-100,:2500],axis=-1),[150,100],preserve_aspect_ratio=False)/255.0)
    elif la == 'R':
        if max_val>=1000:
            return (max_val-tf.image.resize(tf.image.flip_left_right
(tf.expand_dims(image[100:-100,:2500],axis=-1)),[150,100],preserve_aspect_ratio=False))/255.0
        else:
            return (tf.image.resize(tf.image.flip_left_right
(tf.expand_dims(image[100:-100,:2500],axis=-1)),[150,100],preserve_aspect_ratio=False)/255.0)
      #  if max_val>=1000:
      #      return ((max_val-tf.image.resize_with_pad(tf.image.flip_left_right(tf.expand_dims(image,axis=-1)),500,500))/255.0)[:,140:-60,0],(y[int(i.split('/')[-1].split('.')[0])])
      #  else:
       #     return (tf.image.resize_with_pad(tf.image.flip_left_right(tf.expand_dims(image,axis=-1)),500,500)/255.0)[:,140:-60,0],(y[int(i.split('/')[-1].split('.')[0])])



# Create a TensorFlow dataset and map your function to it
dataset= tf.data.Dataset.from_tensor_slices(np.arange((len(list(path_list)))))
dataset = dataset.map(lambda x: tf.numpy_function(gen, inp=[x], Tout=(tf.float32))).batch(1)                

/opt/conda/lib/python3.7/site-packages/tensorflow/python/data/ops/structured_function.py:257: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


In [5]:
def pfbeta_fast(labels, predictions, beta=1):

    pTP = tf.math.reduce_sum(labels * predictions)
    pFP = tf.math.reduce_sum((1-labels) * predictions)
    num_positives = tf.math.reduce_sum((labels))  #  = pTP+pFN

    pPrecision = pTP/(pTP+pFP)
    pRecall = pTP/num_positives

    beta_squared = beta**2

    if (pPrecision > 0 and pRecall > 0):
        pF1 = (1+beta_squared) * pPrecision * pRecall/(beta_squared*pPrecision + pRecall)
        return pF1
    else:
        return 0.0


In [6]:


def mylayer(X,f,size,s,how='same',a=1):
    layer = tf.keras.layers.Conv2D(filters=f,kernel_size=size,strides=s,padding=how)(X)
    layer = tf.keras.layers.BatchNormalization()(layer)
    if a ==1:
        layer = tf.keras.layers.Activation('relu')(layer)
    else:
        layer +=0
    return layer
def add_layer(X_prev,X):
    added = tf.keras.layers.Add()([X_prev,X])
    return added
def conv_connect(X_prev,X,f,s,how='same'):
    layer = tf.keras.layers.Conv2D(filters=f,kernel_size=(1,1),strides=2,padding=how)(X_prev)
    layer = tf.keras.layers.ZeroPadding2D((7,7))(layer)
    layer = tf.keras.layers.BatchNormalization()(layer)
    layer = add_layer(layer,X)
    layer = tf.keras.layers.Activation('relu')(layer)
    return layer

def cnn():
    
    
    pre = tf.keras.applications.convnext.ConvNeXtSmall(
    model_name='convnext_small',
    include_top=False,
    include_preprocessing=True,
    weights=None,
    input_shape=(150,100,3),pooling='avg')
    pre.load_weights('/kaggle/input/naidu-cancer-comp/convnext_small_notop.h5')
    pre.trainable=True
    
    for layer in pre.layers[:-20]:
        layer.trainable=False
    inputs = tf.keras.Input(shape=(150,100,1))
    X = tf.keras.layers.Concatenate()([inputs for i in range(3)])
    X = pre(X)
    #X = tf.keras.layers.Dropout(0.20)(X)
    outputs = tf.keras.layers.Dense(units=1, activation='sigmoid')(X)
    model = tf.keras.Model(inputs=inputs,outputs=outputs)
    model.summary()
    model.compile(loss='BinaryCrossentropy',optimizer = tf.keras.optimizers.Adam(),metrics=['accuracy',pfbeta_fast])
    return model



In [7]:
tf.keras.backend.clear_session()
model = cnn()
model.load_weights('/kaggle/input/naidu-cancer-comp/mlo_l_10e.h5')
#model.fit(mlo_dataset,verbose=True,epochs=7,callbacks=[tf.keras.callbacks.ModelCheckpoint('model.h5')])
 

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 100, 1  0           []                               
                                )]                                                                
                                                                                                  
 concatenate (Concatenate)      (None, 150, 100, 3)  0           ['input_2[0][0]',                
                                                                  'input_2[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 convnext_small (Functional)    (None, 768)          49454688    ['concatenate[0][0]']        

In [8]:
preds = (model.predict(dataset))
test_r = pd.DataFrame({'prediction_id':(test['prediction_id']).unique()})
test_r['cancer']=0
for i, val in enumerate(y.keys()):
        ki = test_r[test_r['prediction_id']==(val.split('_')[0]+'_'+val.split('_')[1])]['prediction_id'].values[0]
        test_r[test_r['prediction_id'] == ki]['cancer']+=preds[check[val],0]
        
    
    

4/4 [==============================] - 7s 1s/step


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
submission = pd.DataFrame({'prediction_id':test_r['prediction_id'],'cancer':test_r['cancer']})
submission.to_csv('submission.csv',index=False)
submission

,prediction_id,cancer
0,10008_L,0
1,10008_R,0
